In [83]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
print("tensorflow version {}".format(tf.VERSION)) # make sure its tensorflow 1.*.*

tensorflow version 1.12.0


In [84]:
#c=a+b
#d=c*a
g0=tf.Graph()
with g0.as_default():
    a = tf.constant([[1],[2]], dtype=tf.float32,name='a_constant')
    b = tf.constant([[2],[3]], dtype=tf.float32,name='b_constant')
    c=tf.add(a,b,name='op')  # c = a + b
    d=tf.multiply(c,a,name='op') #d = c * a
print(a)
print(b)
print(c)
print(d)
#building the graph (the actual computation does not happen here, though its tempting to think of that)
#build the engine

Tensor("a_constant:0", shape=(2, 1), dtype=float32)
Tensor("b_constant:0", shape=(2, 1), dtype=float32)
Tensor("op:0", shape=(2, 1), dtype=float32)
Tensor("op_1:0", shape=(2, 1), dtype=float32)


In [85]:
#computation occurs
#start the engine
sess0=tf.Session(graph=g0)
result=sess0.run([a,c,d])
print(result)

#visualize the graph
writer = tf.summary.FileWriter('.',sess0.graph)
writer.flush()

sess0.close()

[array([[1.],
       [2.]], dtype=float32), array([[3.],
       [5.]], dtype=float32), array([[ 3.],
       [10.]], dtype=float32)]


In [86]:
!pwd

/Users/haobei/Projects/Tutorial


In [87]:
#tensor math matrix
#with "defaultGraph".as_default():
g1=tf.Graph()
a0 = tf.constant(2)
with g1.as_default():
    a = tf.random_normal([2,3], dtype=tf.float32,name='a_random')
    b = tf.constant([[1,1],[1,1],[1,1]], dtype=tf.float32,name='b')
    op = tf.print("value of a",a,name='print')
    var_x = tf.get_variable("var_x", [2,2], dtype=tf.float32, trainable=False,
                            initializer=tf.zeros_initializer)
    c = tf.matmul(a,b,name='matmul') # a @ b
    d = c + var_x #tf.add
print(a0,a0.graph)
print(a,a.graph)
print(b,b.graph)
print(op,op.graph)
print(var_x,var_x.graph)
print(c,c.graph)
print(d,d.graph)

Tensor("Const_1:0", shape=(), dtype=int32) <tensorflow.python.framework.ops.Graph object at 0xb3c5ed0b8>
Tensor("a_random:0", shape=(2, 3), dtype=float32) <tensorflow.python.framework.ops.Graph object at 0xb3ca253c8>
Tensor("b:0", shape=(3, 2), dtype=float32) <tensorflow.python.framework.ops.Graph object at 0xb3ca253c8>
name: "print"
op: "PrintV2"
input: "print_format"
attr {
  key: "output_stream"
  value {
    s: "stderr"
  }
}
<tf.Variable 'var_x:0' shape=(2, 2) dtype=float32_ref> <tensorflow.python.framework.ops.Graph object at 0xb3ca253c8>
Tensor("matmul:0", shape=(2, 2), dtype=float32) <tensorflow.python.framework.ops.Graph object at 0xb3ca253c8>
Tensor("add:0", shape=(2, 2), dtype=float32) <tensorflow.python.framework.ops.Graph object at 0xb3ca253c8>


In [88]:
#engine switch
sess1=tf.Session(graph=g1)

In [89]:
print('print',sess1.run(op))
print('a',sess1.run(a)) #print(a.eval(session=sess))
print('matmul',sess1.run('matmul'))
print('c',sess1.run('matmul:0')) #c

print None
a [[ 2.3015423  0.0855993  1.1671302]
 [ 0.6491126 -2.085204   0.8629421]]
matmul None
c [[ 2.8990245  2.8990245]
 [-2.5520935 -2.5520935]]


In [90]:
#print the design blue print of the engine
writer = tf.summary.FileWriter('.',sess1.graph)
writer.flush()

In [91]:
#facilitate updates of weights of a neural network
with g1.as_default():
    sess1.run(tf.global_variables_initializer())
with g1.as_default():
    print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)) 
    print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

[<tf.Variable 'var_x:0' shape=(2, 2) dtype=float32_ref>]
[]


In [92]:
sess1.close()

In [93]:
x = np.array([[1], [2], [3], [4]],dtype=np.float32)
y = np.array([[4], [3], [2], [1]],dtype=np.float32)

In [94]:
# build default graph
tf.reset_default_graph()
# mini-batch 
x_p = tf.placeholder(dtype=tf.float32,shape=(None,1),name="x_placeholder") # shape is determained at runtime
y_p = tf.placeholder(dtype=tf.float32,shape=(None,1),name="y_placeholder")

W = tf.get_variable("weight",shape=(1,),dtype=tf.float32,
                    initializer=tf.ones_initializer,
                    trainable=True)
b = tf.get_variable("bias",shape=(1,),dtype=tf.float32,
                    initializer=tf.zeros_initializer,
                    trainable=True)
y_hat = W * x_p + b
loss = tf.reduce_mean(tf.square(y_hat - y_p))

In [95]:
sess=tf.Session() # session of the default graph
#print the design blue print of the engine
writer = tf.summary.FileWriter('.',sess.graph)
writer.flush()

In [96]:
gradient = tf.gradients(loss,[W,b])
#print the design blue print of the engine
writer = tf.summary.FileWriter('.',sess.graph)
writer.flush()

In [82]:
sess.run(tf.global_variables_initializer())
W_init = sess.run([W,b])
print(W_init)
prediction = sess.run([y_hat,loss,gradient],feed_dict={x_p: x,y_p: y})
print(prediction)
for i in range(100):
    grad=sess.run(gradient,feed_dict={x_p: x,y_p: y})
    W_old,b_old = sess.run([W,b])
#     sess.run([W.assign_sub(0.1*grad[0]),b.assign_sub(0.1*grad[1])]) what about millions of weights?
    sess.run([var.assign_sub(0.1*gradient) for var,gradient in zip(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES),grad)])
    W_new,b_new = sess.run([W,b])
    print(i,"gradient: ",grad, "\t", W_old,b_old, "==>",W_new,b_new)
print(sess.run([W,b]))

[array([1.], dtype=float32), array([0.], dtype=float32)]
[array([[1.],
       [2.],
       [3.],
       [4.]], dtype=float32), 5.0, [array([5.], dtype=float32), array([0.], dtype=float32)]]
0 gradient:  [5.] 	 [1.] [0.] ==> [0.5] [0.]
1 gradient:  [-2.5] 	 [0.5] [0.] ==> [0.75] [0.25]
2 gradient:  [2.5] 	 [0.75] [0.25] ==> [0.5] [0.325]
3 gradient:  [-0.87499976] 	 [0.5] [0.325] ==> [0.5875] [0.51]
4 gradient:  [1.3624997] 	 [0.5875] [0.51] ==> [0.45125002] [0.61425]
5 gradient:  [-0.15999997] 	 [0.45125002] [0.61425] ==> [0.46725002] [0.765775]
6 gradient:  [0.8376255] 	 [0.46725002] [0.765775] ==> [0.38348746] [0.878995]
7 gradient:  [0.14728665] 	 [0.38348746] [0.878995] ==> [0.3687588] [1.0114523]
8 gradient:  [0.5886433] 	 [0.3687588] [1.0114523] ==> [0.30989447] [1.1247824]
9 gradient:  [0.2723292] 	 [0.30989447] [1.1247824] ==> [0.28266156] [1.2448788]
10 gradient:  [0.46431732] 	 [0.28266156] [1.2448788] ==> [0.23622982] [1.3545723]
11 gradient:  [0.31630898] 	 [0.23622982] [1.

In [ ]:
prediction = sess.run([y_hat,loss,gradient],feed_dict={x_p: x,y_p: y})
print("prediction")
print(prediction[0])
print("target")
print(y)
# 1000 steps

In [ ]:
# single sigmoid neuron prediction on two-dimension data
x = np.array([[0,0],[1,0],[0,1],[1,1]])
y = np.array([[0],[0],[1],[1]])
#tf.sigmoid() tf.reduce_mean()

In [ ]:
#your implementation goes here, add more code cells if necessary

In [ ]:
# visualize the model
vis_inputs = np.random.uniform(size=(10000,2))
predictions = sess.run(y_hat,feed_dict={x_p:vis_inputs})
print(predictions.shape)
colors = list(map(lambda p: 'red' if p < 0.5 else 'blue', predictions))
plt.scatter(vis_inputs[:,0], vis_inputs[:,1], 1, colors)

In [ ]:
# single sigmoid neuron prediction on () two-dimension data
x = np.array([[0,0],[1,0],[0,1],[1,1]])
y = np.array([[0],[1],[1],[0]])

In [ ]:
# visualize the model
vis_inputs = np.random.uniform(size=(10000,2))
predictions = sess.run(y_hat,feed_dict={x_p:vis_inputs})
print(predictions.shape)
colors = list(map(lambda p: 'red' if p < 0.5 else 'blue', predictions))
plt.scatter(vis_inputs[:,0], vis_inputs[:,1], 1, colors)